In [ ]:
import numpy as np 
import pandas as pd 
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('F:/fall 2022/IST 707/homework/week 8/kaggle_digits/Kaggle-digit-train.csv').astype(float)
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: float64(785)
memory usage: 251.5 MB


# split the data

In [ ]:
X = df.drop('label', axis = 1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

# simple experiment of parameters

In [ ]:
# import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# instantiate the DecisionTreeClassifier model with criterion entropy
clf_en = DecisionTreeClassifier(criterion='entropy', splitter='best',max_depth=8, random_state=1)

# fit the model
clf_en.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=8, random_state=1)

In [ ]:
#Predict the Test set results 
y_pred_en = clf_en.predict(X_test)

In [ ]:
#Check accuracy score 
from sklearn.metrics import accuracy_score
print('Model accuracy score with criterion entropy: {0:0.4f}'. format(accuracy_score(y_test, y_pred_en)))

Model accuracy score with criterion entropy: 0.8310


In [ ]:
#Compare the train-set and test-set accuracy
y_pred_train_en = clf_en.predict(X_train)

y_pred_train_en

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_en)))
print('Testing-set accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_en)))

Training-set accuracy score: 0.8595
Testing-set accuracy score: 0.8310


In [ ]:
#confustion matrix
# Print the Confusion Matrix and slice it into four pieces

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_en)

print('Confusion matrix\n\n', cm)

Confusion matrix

 [[1134    0   21   13    5   20   22    4   18   10]
 [   0 1267   16   33    7    4    5   13   49    1]
 [  31   34  987   22   34   10   40   45   34   20]
 [  14   11   44  920   19  136   11   24   89   53]
 [   3    3   20   10  973   14   50   31   21  107]
 [  52    6   24  114   38  772   48   12   27   18]
 [  17    4   65   20   46   39 1043   11   10    3]
 [   6   28   52   14   28    8    5 1057    4   83]
 [   0   33   53   54   38   32   51   11  929   48]
 [   4   11   15   41  144   25   15   36   47  907]]


In [ ]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_en))

              precision    recall  f1-score   support

         0.0       0.90      0.91      0.90      1247
         1.0       0.91      0.91      0.91      1395
         2.0       0.76      0.79      0.77      1257
         3.0       0.74      0.70      0.72      1321
         4.0       0.73      0.79      0.76      1232
         5.0       0.73      0.69      0.71      1111
         6.0       0.81      0.83      0.82      1258
         7.0       0.85      0.82      0.84      1285
         8.0       0.76      0.74      0.75      1249
         9.0       0.73      0.73      0.73      1245

    accuracy                           0.79     12600
   macro avg       0.79      0.79      0.79     12600
weighted avg       0.79      0.79      0.79     12600



In [ ]:
# cv accuracy
from sklearn.model_selection import cross_val_score
cross_val_score(clf_en, X, y, cv=3)

array([0.79092857, 0.79721429, 0.8055    ])

# Selection of the optimal model

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import tree

In [ ]:
#set the range of parameter
parameters = {'criterion':('gini', 'entropy'), 
              'min_samples_split':[2,3,4,5], 
              'max_depth':[3,4,5,6,7],
             }

In [ ]:
tr = tree.DecisionTreeClassifier()
gsearch = GridSearchCV(tr, parameters,cv=3,n_jobs=-1)
gsearch.fit(X_train, y_train)
model = gsearch.best_estimator_
model

DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_split=3)

In [ ]:
# report cv 
gsearch.cv_results_
scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
scores

array([0.79223214, 0.78875   , 0.79616071])

# application of the optimal model

In [ ]:
import time
start = time.time()
# instantiate the DecisionTreeClassifier model with criterion entropy
clf_en = DecisionTreeClassifier(criterion='entropy', splitter='best',max_depth=7, min_samples_split=3, random_state=1)

# fit the model
clf_en.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_split=3,
                       random_state=1)

In [ ]:
#Predict the Test set results 
y_pred_en = clf_en.predict(X_test)

In [ ]:
#Check accuracy score 
from sklearn.metrics import accuracy_score
print('Model accuracy score with criterion entropy: {0:0.4f}'. format(accuracy_score(y_test, y_pred_en)))

Model accuracy score with criterion entropy: 0.7928


In [ ]:
#Check for overfitting and underfitting
# print the scores on training and test set
#Compare the train-set and test-set accuracy
y_pred_train_en = clf_en.predict(X_train)

y_pred_train_en

print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train_en)))
print('Testing-set accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_en)))

Training-set accuracy score: 0.8116
Testing-set accuracy score: 0.7928


In [ ]:
#confustion matrix
# Print the Confusion Matrix and slice it into four pieces

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_en)

print('Confusion matrix\n\n', cm)

Confusion matrix

 [[1134    0   21   13    5   20   22    4   18   10]
 [   0 1267   16   33    7    4    5   13   49    1]
 [  31   34  987   22   34   10   40   45   34   20]
 [  14   11   44  920   19  136   11   24   89   53]
 [   3    3   20   10  973   14   50   31   21  107]
 [  52    6   24  114   38  772   48   12   27   18]
 [  17    4   65   20   46   39 1043   11   10    3]
 [   6   28   52   14   28    8    5 1057    4   83]
 [   0   33   53   54   38   32   51   11  929   48]
 [   4   11   15   41  144   25   15   36   47  907]]


In [ ]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_en))

              precision    recall  f1-score   support

         0.0       0.90      0.91      0.90      1247
         1.0       0.91      0.91      0.91      1395
         2.0       0.76      0.79      0.77      1257
         3.0       0.74      0.70      0.72      1321
         4.0       0.73      0.79      0.76      1232
         5.0       0.73      0.69      0.71      1111
         6.0       0.81      0.83      0.82      1258
         7.0       0.85      0.82      0.84      1285
         8.0       0.76      0.74      0.75      1249
         9.0       0.73      0.73      0.73      1245

    accuracy                           0.79     12600
   macro avg       0.79      0.79      0.79     12600
weighted avg       0.79      0.79      0.79     12600



In [ ]:
# cv accuracy
from sklearn.model_selection import cross_val_score
cross_val_score(clf_en, X, y, cv=3)
end = time.time()
print(end - start)

34.00476050376892
